## Импорт библиотек

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', None)

## Загрузка данных

In [3]:
file_path = '/data/share/lab04data/lab04_train.csv'
df = pd.read_csv(file_path, sep=',')
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.head()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,479990,0,0.000000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0.0,1.906969e+04,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,0.0,0.000000,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.981818,0.926472,0.000000,324.0,NaN,0.0,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.139327,0.000000,NaN,NaN,NaN,0.0,0.244042,NaN,0.144300,0.000000,NaN,0.000000,0.351852,0.166667,0.000000,0.329321,0.0,NaN,45.00,NaN,45.00,0.167901,0.000000,NaN,0.0,0.303750,102,NaN,NaN,5.480001,0.0,0.000000,NaN,0.000000,NaN,0.305106,NaN,0.345455,NaN,0.0,1.0,0.0,NaN,0.003524,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,450480,0,0.000000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.027168,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0.0,5.951209e+03,NaN,NaN,NaN,0.048746,офис-менеджер,NaN,0.030303,NaN,NaN,NaN,0.0,0.0,0.030303,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.090909,0.292473,0.000000,360.0,NaN,0.0,NaN,0.000000,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.163876,0.000000,NaN,NaN,NaN,0.0,0.465244,1.000000,0.203484,0.000000,1.000000,0.000000,0.333333,0.333333,0.000000,0.333333,0.0,NaN,45.00,NaN,45.00,0.333333,0.000000,NaN,0.0,0.504160,102,NaN,NaN,6.786453,0.0,0.000000,NaN,0.000000,NaN,0.368799,NaN,0.454545,NaN,0.0,1.0,0.0,NaN,0.000618,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,473954,0,0.000000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.114951,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0.0,3.774583e+03,0.250000,NaN,NaN,0.004519,консультант,NaN,0.363636,NaN,NaN,0.279487,0.0,0.0,0.136364,NaN,0.0,1.0,NaN,0.0,NaN,NaN,0.0,0.136364,0.533251,0.000000,600.0,1.000000,0.0,1.000000,0.000000,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.073887,0.000000,NaN,NaN,NaN,0.0,0.490018,0.375000,0.090256

## Предобработка данных

Здесь должен быть основной объем работы, но в целях проверки baseline модели была проведена попытка загнать все в бустинг и посмотреть качество, в итоге чеккер был пройден

## Выделим категориальные переменные

In [5]:
df.loc[:,df.dtypes.loc[lambda x: x=='object'].index]

,CLNT_TRUST_RELATION,APP_MARITAL_STATUS,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,APP_EDUCATION,APP_TRAVEL_PASS,APP_CAR,APP_POSITION_TYPE,APP_EMP_TYPE,APP_COMP_TYPE,PACK
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
1,NaN,NaN,NaN,NaN,офис-менеджер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
2,NaN,NaN,NaN,NaN,консультант,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K01
4,NaN,NaN,NaN,NaN,экономист,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K01
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
6,NaN,NaN,NaN,NaN,Младший научный сотрудник,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O01
7,NaN,M,RENT,NaN,ДИРЕКТОР,Y,UH,N,N,MANAGER,PRIVATE,PRIVATE,O01
8,FRIEND,V,NaN,MANAGER,СПЕЦИАЛИСТ,NaN,H,NaN,NaN,SPECIALIST,PRIVATE,PRIVATE,K01
9,NaN,NaN,NaN,NaN,глав.бухгалтер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K01


In [6]:
cat_feats=list(df.dtypes.loc[lambda x: x=='object'].index)

In [7]:
#Найдем номера столбцов категориальных признаков
cat_feats_ind = []
for col in cat_feats:
    df[col] = df[col].astype('str')
    cat_feats_ind.append(df.columns.get_loc(col))

In [8]:
#Индексы категориальных переменных
cat_feats_ind

[13, 19, 24, 25, 27, 28, 30, 36, 39, 42, 53, 66, 88]

## Обучение алгоритма

In [9]:
train_data = Pool(data=df.drop(['TARGET'], axis=1),
                  label=df['TARGET'],
                  cat_features=cat_feats_ind)

In [10]:
model = CatBoostClassifier()
model.fit(train_data)

Learning rate set to 0.085778
0:	learn: 0.5941744	total: 397ms	remaining: 6m 37s
1:	learn: 0.5177956	total: 627ms	remaining: 5m 12s
2:	learn: 0.4583819	total: 844ms	remaining: 4m 40s
3:	learn: 0.4091044	total: 1.09s	remaining: 4m 30s
4:	learn: 0.3764078	total: 1.32s	remaining: 4m 22s
5:	learn: 0.3511330	total: 1.58s	remaining: 4m 22s
6:	learn: 0.3280497	total: 1.83s	remaining: 4m 19s
7:	learn: 0.3115749	total: 2.05s	remaining: 4m 14s
8:	learn: 0.2985425	total: 2.27s	remaining: 4m 9s
9:	learn: 0.2874671	total: 2.48s	remaining: 4m 5s
10:	learn: 0.2797344	total: 2.72s	remaining: 4m 4s
11:	learn: 0.2738044	total: 2.97s	remaining: 4m 4s
12:	learn: 0.2674131	total: 3.22s	remaining: 4m 4s
13:	learn: 0.2614348	total: 3.46s	remaining: 4m 3s
14:	learn: 0.2573736	total: 3.7s	remaining: 4m 3s
15:	learn: 0.2536429	total: 3.96s	remaining: 4m 3s
16:	learn: 0.2503115	total: 4.21s	remaining: 4m 3s
17:	learn: 0.2478740	total: 4.42s	remaining: 4m 1s
18:	learn: 0.2459131	total: 4.67s	remaining: 4m 1s
19:	

158:	learn: 0.2165084	total: 41.8s	remaining: 3m 41s
159:	learn: 0.2164732	total: 42s	remaining: 3m 40s
160:	learn: 0.2164343	total: 42.3s	remaining: 3m 40s
161:	learn: 0.2163869	total: 42.5s	remaining: 3m 39s
162:	learn: 0.2163672	total: 42.8s	remaining: 3m 39s
163:	learn: 0.2163300	total: 43s	remaining: 3m 39s
164:	learn: 0.2163044	total: 43.3s	remaining: 3m 38s
165:	learn: 0.2162660	total: 43.5s	remaining: 3m 38s
166:	learn: 0.2162094	total: 43.7s	remaining: 3m 38s
167:	learn: 0.2161831	total: 44s	remaining: 3m 37s
168:	learn: 0.2161525	total: 44.2s	remaining: 3m 37s
169:	learn: 0.2161075	total: 44.5s	remaining: 3m 37s
170:	learn: 0.2160456	total: 44.7s	remaining: 3m 36s
171:	learn: 0.2160116	total: 44.9s	remaining: 3m 36s
172:	learn: 0.2159780	total: 45.2s	remaining: 3m 35s
173:	learn: 0.2159177	total: 45.4s	remaining: 3m 35s
174:	learn: 0.2158876	total: 45.6s	remaining: 3m 34s
175:	learn: 0.2158291	total: 45.9s	remaining: 3m 34s
176:	learn: 0.2158065	total: 46.1s	remaining: 3m 34s

314:	learn: 0.2116572	total: 1m 20s	remaining: 2m 54s
315:	learn: 0.2116043	total: 1m 20s	remaining: 2m 54s
316:	learn: 0.2115786	total: 1m 20s	remaining: 2m 54s
317:	learn: 0.2115637	total: 1m 21s	remaining: 2m 54s
318:	learn: 0.2115277	total: 1m 21s	remaining: 2m 53s
319:	learn: 0.2115187	total: 1m 21s	remaining: 2m 53s
320:	learn: 0.2115064	total: 1m 21s	remaining: 2m 53s
321:	learn: 0.2114875	total: 1m 22s	remaining: 2m 53s
322:	learn: 0.2114478	total: 1m 22s	remaining: 2m 52s
323:	learn: 0.2114295	total: 1m 22s	remaining: 2m 52s
324:	learn: 0.2114221	total: 1m 22s	remaining: 2m 52s
325:	learn: 0.2114077	total: 1m 23s	remaining: 2m 51s
326:	learn: 0.2113819	total: 1m 23s	remaining: 2m 51s
327:	learn: 0.2113471	total: 1m 23s	remaining: 2m 51s
328:	learn: 0.2113167	total: 1m 23s	remaining: 2m 51s
329:	learn: 0.2112930	total: 1m 24s	remaining: 2m 50s
330:	learn: 0.2112437	total: 1m 24s	remaining: 2m 50s
331:	learn: 0.2112155	total: 1m 24s	remaining: 2m 50s
332:	learn: 0.2111903	total:

466:	learn: 0.2084381	total: 1m 59s	remaining: 2m 16s
467:	learn: 0.2084345	total: 1m 59s	remaining: 2m 16s
468:	learn: 0.2083903	total: 1m 59s	remaining: 2m 15s
469:	learn: 0.2083865	total: 2m	remaining: 2m 15s
470:	learn: 0.2083839	total: 2m	remaining: 2m 15s
471:	learn: 0.2083777	total: 2m	remaining: 2m 15s
472:	learn: 0.2083581	total: 2m	remaining: 2m 14s
473:	learn: 0.2083479	total: 2m 1s	remaining: 2m 14s
474:	learn: 0.2083416	total: 2m 1s	remaining: 2m 14s
475:	learn: 0.2083239	total: 2m 1s	remaining: 2m 13s
476:	learn: 0.2083042	total: 2m 1s	remaining: 2m 13s
477:	learn: 0.2082970	total: 2m 2s	remaining: 2m 13s
478:	learn: 0.2082835	total: 2m 2s	remaining: 2m 13s
479:	learn: 0.2082651	total: 2m 2s	remaining: 2m 12s
480:	learn: 0.2082554	total: 2m 2s	remaining: 2m 12s
481:	learn: 0.2082445	total: 2m 3s	remaining: 2m 12s
482:	learn: 0.2082267	total: 2m 3s	remaining: 2m 11s
483:	learn: 0.2082065	total: 2m 3s	remaining: 2m 11s
484:	learn: 0.2081879	total: 2m 3s	remaining: 2m 11s
48

620:	learn: 0.2060983	total: 2m 38s	remaining: 1m 36s
621:	learn: 0.2060678	total: 2m 39s	remaining: 1m 36s
622:	learn: 0.2060526	total: 2m 39s	remaining: 1m 36s
623:	learn: 0.2060436	total: 2m 39s	remaining: 1m 36s
624:	learn: 0.2060339	total: 2m 39s	remaining: 1m 35s
625:	learn: 0.2060161	total: 2m 40s	remaining: 1m 35s
626:	learn: 0.2060039	total: 2m 40s	remaining: 1m 35s
627:	learn: 0.2059890	total: 2m 40s	remaining: 1m 35s
628:	learn: 0.2059805	total: 2m 40s	remaining: 1m 34s
629:	learn: 0.2059619	total: 2m 41s	remaining: 1m 34s
630:	learn: 0.2059550	total: 2m 41s	remaining: 1m 34s
631:	learn: 0.2059497	total: 2m 41s	remaining: 1m 34s
632:	learn: 0.2059232	total: 2m 41s	remaining: 1m 33s
633:	learn: 0.2059175	total: 2m 42s	remaining: 1m 33s
634:	learn: 0.2059004	total: 2m 42s	remaining: 1m 33s
635:	learn: 0.2058832	total: 2m 42s	remaining: 1m 33s
636:	learn: 0.2058710	total: 2m 42s	remaining: 1m 32s
637:	learn: 0.2058534	total: 2m 42s	remaining: 1m 32s
638:	learn: 0.2058353	total:

774:	learn: 0.2039419	total: 3m 17s	remaining: 57.3s
775:	learn: 0.2039239	total: 3m 17s	remaining: 57s
776:	learn: 0.2039185	total: 3m 17s	remaining: 56.7s
777:	learn: 0.2038877	total: 3m 17s	remaining: 56.5s
778:	learn: 0.2038700	total: 3m 18s	remaining: 56.2s
779:	learn: 0.2038528	total: 3m 18s	remaining: 56s
780:	learn: 0.2038342	total: 3m 18s	remaining: 55.7s
781:	learn: 0.2038257	total: 3m 18s	remaining: 55.5s
782:	learn: 0.2038077	total: 3m 19s	remaining: 55.2s
783:	learn: 0.2038023	total: 3m 19s	remaining: 54.9s
784:	learn: 0.2037986	total: 3m 19s	remaining: 54.7s
785:	learn: 0.2037799	total: 3m 19s	remaining: 54.4s
786:	learn: 0.2037656	total: 3m 20s	remaining: 54.2s
787:	learn: 0.2037567	total: 3m 20s	remaining: 53.9s
788:	learn: 0.2037280	total: 3m 20s	remaining: 53.6s
789:	learn: 0.2037174	total: 3m 20s	remaining: 53.4s
790:	learn: 0.2037158	total: 3m 21s	remaining: 53.1s
791:	learn: 0.2037067	total: 3m 21s	remaining: 52.9s
792:	learn: 0.2037046	total: 3m 21s	remaining: 52.

930:	learn: 0.2020157	total: 3m 56s	remaining: 17.5s
931:	learn: 0.2020032	total: 3m 56s	remaining: 17.3s
932:	learn: 0.2019970	total: 3m 57s	remaining: 17s
933:	learn: 0.2019873	total: 3m 57s	remaining: 16.8s
934:	learn: 0.2019668	total: 3m 57s	remaining: 16.5s
935:	learn: 0.2019472	total: 3m 57s	remaining: 16.3s
936:	learn: 0.2019357	total: 3m 58s	remaining: 16s
937:	learn: 0.2019289	total: 3m 58s	remaining: 15.8s
938:	learn: 0.2019167	total: 3m 58s	remaining: 15.5s
939:	learn: 0.2019026	total: 3m 58s	remaining: 15.3s
940:	learn: 0.2018909	total: 3m 59s	remaining: 15s
941:	learn: 0.2018823	total: 3m 59s	remaining: 14.7s
942:	learn: 0.2018725	total: 3m 59s	remaining: 14.5s
943:	learn: 0.2018649	total: 3m 59s	remaining: 14.2s
944:	learn: 0.2018521	total: 4m	remaining: 14s
945:	learn: 0.2018329	total: 4m	remaining: 13.7s
946:	learn: 0.2018241	total: 4m	remaining: 13.5s
947:	learn: 0.2018228	total: 4m	remaining: 13.2s
948:	learn: 0.2018126	total: 4m 1s	remaining: 13s
949:	learn: 0.201797

In [11]:
y_pred = model.predict_proba(train_data)[:,1]

## Оценка качества модели на train

In [116]:
roc_auc_score(df['TARGET'], y_pred)

0.8698726007703885

## Скоринг теста

Повторим пайплайн для train

In [117]:
file_path = '/data/share/lab04data/lab04_test.csv'
df = pd.read_csv(file_path, sep=',')
df = df.drop('Unnamed: 0', axis=1)

In [118]:
for col in cat_feats:
    df[col] = df[col].astype('str')

In [119]:
y_pred_test = model.predict_proba(df)[:,1]

## Сохранение результата

In [123]:
pd.concat([df['ID'], pd.Series(y_pred_test)], axis=1)\
              .rename(columns={'ID':'id',0:'target'}).to_csv('lab04.csv', sep='\t', index=False)

В итоге получаем проходной балл с ROC_AUC = 0.8561